In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from tabpfn import TabPFNClassifier

train = pd.read_csv('/kaggle/input/playground-series-s5e11/train.csv', index_col='id')
X = pd.read_csv('/kaggle/input/loan-prediction-dataset-2025/loan_dataset_20000.csv')[train.columns]
y = X.pop('loan_paid_back').astype(int)
cat_cols = [c for c in X.columns if X[c].dtype=='object']

model = make_pipeline(
    make_column_transformer(
        (OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols),
        remainder='passthrough', verbose_feature_names_out=False
    ).set_output(transform='pandas'),
    TabPFNClassifier(
        ignore_pretraining_limits=True,
        model_path='tabpfn-v2.5-classifier-v2.5_default.ckpt',
        device=['cuda:0', 'cuda:1']
    )
)

kfold = StratifiedKFold(5, shuffle=True, random_state=0)
scores = cross_val_score(
    model, X, y, cv=kfold, scoring='roc_auc',
    n_jobs=1
)
print(F'{scores.mean():.5f} ± {scores.std():.5f}')